# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
#используемые библиотеки

import pandas as pd

In [2]:
#загрузка данных

from io import BytesIO
import requests
spreadsheet_id = '1BcksdDMjtj_VVBuAznyyzSM_ru1LgXF1HkfkliNHotA'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
data = pd.read_csv(BytesIO(r.content))
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


    Для удобства и во избежание дальнейших ошибок, переведем весь тест в нижний регистр.

In [3]:
data['education'] = data['education'].str.lower() #перевод в нижний регистр
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выводы**

При первом просмотре в данных обнаружены следующие недочеты:
1. Отрицательные значения, нецелые в столбце 'days_employed'
2. Разный регистр в столбце 'education'

# Шаг 2. Предобработка данных

## Обработка пропусков

In [4]:
print('Количество пропусков:')
print(data.isnull().sum())

Количество пропусков:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Вывод**

1. Пропущенные значения есть в столбцах 'days_employed' и 'total_income'. Их колличество слишком велико, чтобы просто удалить эти строки.
   Пропущенных значений около 10%, вероятно, имеет место систематическая ошибка. Чтобы заполнить значения дохода, берем медианное значение, чтобы исключить вероятность большого разброса в значениях

2. Так как в данной задаче трудовой стаж не явяется важной переменной, заменим пустые значения на выбранное значение (1000). Если позже оно нам встретится, мы точно будем знать откуда оно, так как в отличие от остальных значений этого столбца это значение круглое и целое.
    Также смущает, что значение трудового стажа, указанного в днях, не целочисленно.

In [5]:
total_income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(total_income_median) #замена отсутсвующих значений в 'total_income' на медианное
print('Количество пропусков в "total_income" после замены:', data['total_income'].isnull().sum())

data['days_employed'] = data['days_employed'].fillna(1000)
print()
print('Количество пропусков в "days_employed" после замены:', data['days_employed'].isnull().sum())

Количество пропусков в "total_income" после замены: 0

Количество пропусков в "days_employed" после замены: 0


### Замена типа данных

In [6]:
data['days_employed'] = pd.to_numeric(data['days_employed'], errors='coerce') #перевод значений в тип float
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

1. Для столбца 'days_employed' выполнен перевод в числовой тип (тип столбца стал float64).

### Обработка дубликатов

In [7]:
print('Колличество дубликатов изначально:', data.duplicated().sum()) 
duplicat = (data.duplicated().sum()/data['days_employed'].count())*100
print('Доля дубликатов от общего числа данных, %:', duplicat) 
data = data.drop_duplicates().reset_index(drop=True) #так как дуюликатов меньше 1% их удаление никак не скажется на статистике
print('Колличество дубликатов после удаления:', data.duplicated().sum())

Колличество дубликатов изначально: 71
Доля дубликатов от общего числа данных, %: 0.32984901277584205
Колличество дубликатов после удаления: 0


**Вывод**

Количество дубликатов очень мало, поэтому их удаление не скажется на финальных данных.

### Лемматизация

Для того, чтобы определить самые популярные цели кредитования, необходимо обработать столбец "purpose". 
Для определения цели кредитования используем метод лемматизации. Этот метод позволяет выделить лемму (начальную форму) слова и работать с ней.

In [8]:
from pymystem3 import Mystem #импортируем библиотеку
m = Mystem()

data_purpose_unique = data['purpose'].unique() #выделяем уникальные значения столбца 'purpose'
lemm_list=[]

for element in data_purpose_unique:
    lemm_list.extend((m.lemmatize(element))) #проводим лемматизацию выделенных значений

from collections import Counter
lemm_list_counter = (Counter(lemm_list)) #далее считаем количество их упоминаний
display(lemm_list_counter)
#display(lemm_list)

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '\n': 38,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

**Вывод**

1. На основе прведенной лемматизации видно, что распространенные цели кредита это: недвижимость(жилье (7 упоминаний), недвижимость(10)), образование (9), автомобиль (9).
    Эти категории и будут приняты за основные при категоризации данных.

### Категоризация данных

Так как основной целью задачи является поиск зависимости между наличием детей и возвратом кредита, было принято решение категоризировать клиентов по количеству детей.

In [9]:
#категоризировать данные предлагается по количеству детей

def function_group(children): #создаем функцию для категоризации
    
    if children == 0:
        return 'бездетный'
    
    if children <= 3:
        return 'с детьми' 
    return 'многодетный'

data['purpose_category'] = data['children'].apply(function_group) #применяем функцию на весь датафрейм
#print(data.head())
print(data['purpose_category'].value_counts()) #подсчитываем кол-во клиентов в каждой категории

бездетный      14091
с детьми        7237
многодетный      126
Name: purpose_category, dtype: int64


**Вывод**

1. После категоризации видно, что больше всего клиентов без детей, далее идут клиенты с детьми, и совсем не много многодетных клиентов.

## Шаг 3. Ответьте на вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Для ответа на это вопрос посчитаем какую долю от должников составляют клиенты с детьми и без детей. 

In [10]:
debtors_without_children = data[(data['children']==0) & (data['debt']!=0)] 
#с помощью метода .loc создаем таблицу, в которой логическими условиями выделяем клиентов без детей, но с долгами 
data_without_children = data[data['children']==0]
procent_without_children = len(debtors_without_children)/len(data_without_children)
print('Количество должников без детей:', len(debtors_without_children)) #методом len() определяем количество нужных нам значений
print('Доля должников с детьми от общего кол-ва должников, %:', '{:.1%}'.format(procent_without_children))
print()

debtors_with_children = data[(data['children']!=0) & (data['debt']!=0)] #дублирует принцип для клиентов с детьми и с долгами
print('Количество должников с детьми:', len(debtors_with_children))
data_with_children = data[data['children']!=0]
procent_with_children = len(debtors_with_children)/len(data_with_children)
print('Доля должников с детьми от общего кол-ва должников, %:', '{:.1%}'.format(procent_with_children))

Количество должников без детей: 1063
Доля должников с детьми от общего кол-ва должников, %: 7.5%

Количество должников с детьми: 678
Доля должников с детьми от общего кол-ва должников, %: 9.2%


**Вывод**

Количество должников c детьми на 1,7% больше, чем должников без детей. Следовательно, клиенты с детьми будут более рисковой группой.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Так как долги в столбце 'debt' указаны как логический тип, среднее арифметическое и будет процентом должников.

In [11]:
print(data.groupby('family_status')['debt'].mean()) #сгруппируем данные по семейному положению и найдем среднее арифметическое

family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
Name: debt, dtype: float64


**Вывод**

1. Самая рисковая группа с 9,8% должников - Не женат / не замужем.
2. Самая безопасная группа с 6,6% должников - вдовец / вдова.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Для того, чтобы ответить на этот вопрос, проведем категоризацию клиентов по доходам. Низкий дохад был принят ниже медианного, средний - от медианного до третьего квартиля и высокий доход - выше третьего квартиля.

In [12]:
total_income_mean = data['total_income'].mean() #найдено среднее значение дохода
#print(data.describe()) #выводим метод 'describe' 
data_75 = 195814

def categorize(row):
    income = row['total_income']
    if income <= total_income_mean:
        return 'низкий'
    if income <= data_75:
        return 'средний'
    return 'высокий'

data['categorize'] = data.apply(categorize, axis=1)
#display(data.head())

print(data.groupby('categorize')['debt'].mean()) #группируем по уровню дозода и находим 

categorize
высокий    0.071402
низкий     0.083267
средний    0.091145
Name: debt, dtype: float64


**Выводы**

1. Рисковой группой с процентом должников 9,1% являются клиенты со средним доходом. 
2. Самая безопасная группа (7,1% должников) - клиенты с высоким доходом.

- **Как разные цели кредита влияют на его возврат в срок?**

На основании проведенной выше лемматизации, выделяем самые популярные цели кредитования. После этого рассчитываем долю должников для каждой цели кредита.

In [ ]:
def categorize_purpose(purpose):
    row_lemmatized = m.lemmatize(purpose)

    if "автомобиль" in row_lemmatized:
        return "автомобиль"
    elif "недвижимость" in row_lemmatized or "жилье" in row_lemmatized:
        return "жилье"

data['purpose'] = data['purpose'].apply(categorize_purpose) 

data['purpose'] = data['purpose'].fillna("другое") #заполняем отсутствующие значения

In [ ]:
print(data.groupby('purpose')['debt'].mean())

**Вывод**

1. Cамой надежной целью кредитования является недвижимость (7,2% должников).

2. Самая рисковая категория - автомобиль (9,4% должников).

## Общий вывод

По запросу заказчика были проверены несколько теорий по о влиянии различных факторов на вероятность выплаты кредита в срок. В рамках данной работы были сделаны следующие выводы: 

    1. Клиенты без детей более платежеспособны, чем клиенты с детьми. Среди должников клиенты без детей составляют 7,5%, в то вреям как клиенты с детьми - 9,2%.
    2. Выявлена зависимость между семейный положением и возвратом кредита в срок. Самая надежная группа заемщиков - вдова/вдовец (6,6% среди должников), самая рисковая группа - не замужем/не женат (9,8% среди должников).
    3. Закономерность для уровня доходов следующая: высокий доход - 7,1% должников, низкий доход - 8,3% должников, средний доход - 9,1%.
    4. Самой рисковой целью кредитовая является автомобиль (9,4% должников), в то время как самой надежной - недвижимость (7,2% должников).